In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import efficientnet
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------------
# 1. Load the eurosat/rgb Dataset
# ---------------------------------------------------
# Create train/validation split
(dataset_train, dataset_test), info = tfds.load(
    'eurosat/rgb',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# Number of classes in eurosat/rgb
num_classes = info.features['label'].num_classes

# ---------------------------------------------------
# 2. Preprocess Data for EfficientNet
# ---------------------------------------------------
IMG_SIZE = 224
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_efficientnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = efficientnet.preprocess_input(image)  # Use efficientnet's preprocess_input
    return image, tf.one_hot(label, num_classes)

# Create tf.data pipelines
train_ds_eff = (dataset_train
                .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
                .batch(batch_size)
                .prefetch(AUTOTUNE))

test_ds_eff = (dataset_test
               .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
               .batch(batch_size)
               .prefetch(AUTOTUNE))

# ---------------------------------------------------
# 3. Define EfficientNetB0 Model
# ---------------------------------------------------
def create_efficientnet(input_shape, num_classes):
    base_model = efficientnet.EfficientNetB0(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="EfficientNetB0")
    return model

# ---------------------------------------------------
# 4. Compile and Train the Model
# ---------------------------------------------------
# EfficientNetB0 Model
eff_model = create_efficientnet((IMG_SIZE, IMG_SIZE, 3), num_classes)
eff_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning EfficientNetB0 on eurosat/rgb ---")
history_eff = eff_model.fit(train_ds_eff, epochs=50, validation_data=test_ds_eff, verbose=1)

# Evaluate EfficientNetB0
loss_eff, acc_eff = eff_model.evaluate(test_ds_eff, verbose=0)
print(f"EfficientNetB0 Final Accuracy: {acc_eff:.4f}")



--- Fine-Tuning EfficientNetB0 on eurosat/rgb ---
Epoch 1/50
675/675 ━━━━━━━━━━━━━━━━━━━━ 60s 60ms/step - accuracy: 0.4494 - loss: 1.7511 - val_accuracy: 0.8693 - val_loss: 0.6920
Epoch 2/50
675/675 ━━━━━━━━━━━━━━━━━━━━ 60s 41ms/step - accuracy: 0.8528 - loss: 0.6532 - val_accuracy: 0.9011 - val_loss: 0.4294
Epoch 3/50
675/675 ━━━━━━━━━━━━━━━━━━━━ 27s 41ms/step - accuracy: 0.8896 - loss: 0.4446 - val_accuracy: 0.9157 - val_loss: 0.3327
Epoch 4/50
675/675 ━━━━━━━━━━━━━━━━━━━━ 46s 48ms/step - accuracy: 0.9034 - loss: 0.3642 - val_accuracy: 0.9237 - val_loss: 0.2814
Epoch 5/50
675/675 ━━━━━━━━━━━━━━━━━━━━ 36s 40ms/step - accuracy: 0.9107 - loss: 0.3142 - val_accuracy: 0.9294 - val_loss: 0.2493
Epoch 6/50
675/675 ━━━━━━━━━━━━━━━━━━━━ 41s 40ms/step - accuracy: 0.9154 - loss: 0.2870 - val_accuracy: 0.9335 - val_loss: 0.2278
Epoch 7/50
675/675 ━━━━━━━━━━━━━━━━━━━━ 27s 41ms/step - accuracy: 0.9207 - loss: 0.2630 - val_accuracy: 0.9378 - val_loss: 0.2116
Epoch 8/50
675/675 ━━━━━━━━━━━━━━━━━━━━